In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import collections

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from scipy.ndimage.filters import gaussian_filter
import cv2

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# GPU Device
gpu_id = 1
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 1: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'
target_dir = '/media/data2/dataset/GAN_ImageData/StarGAN_128/'

In [4]:
pretrained = './log/pggan/128/b0/aug/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 500
start_epoch = 0
train_batch = 200
test_batch = 200
lr = 0.001
schedule = [50, 250, 500, 750]
momentum = 0.1
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/pggan/128/b0/to_star/2000shot/general' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# cutmix
cm_prob = 0.2
cm_beta = 1.0

# augmentation
blur_prob = 0.2
blog_sig = 0.5
jpg_prob = 0.2

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
def data_augment(img):
    img = np.array(img)

    if random.random() < blur_prob:
        sig = np.random.uniform(0.0, 3.0)
        gaussian_blur(img, sig)

    if random.random() < jpg_prob:
        qual = np.random.uniform(30.0, 100.0)
        img = cv2_jpg(img, qual)

    return Image.fromarray(img)


def gaussian_blur(img, sigma):
    gaussian_filter(img[:,:,0], output=img[:,:,0], sigma=sigma)
    gaussian_filter(img[:,:,1], output=img[:,:,1], sigma=sigma)
    gaussian_filter(img[:,:,2], output=img[:,:,2], sigma=sigma)


def cv2_jpg(img, compress_val):
    img_cv2 = img[:,:,::-1]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compress_val]
    result, encimg = cv2.imencode('.jpg', img_cv2, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg[:,:,::-1]

In [8]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Dataset

In [9]:
train_dir = os.path.join(source_dir, 'star/2000_shot_only')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.Lambda(lambda img: data_augment(img)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [10]:
student_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/pggan/128/b0/aug/checkpoint.pth.tar'


In [11]:
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in student_model.parameters())/1000000.0))

    Total params: 4.01M


In [12]:
for idx, param in enumerate(student_model.parameters()):
    param.reqiures_grad = False
    if idx == 182:
        break

In [13]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
# optimizer = optim.Adam(student_model.parameters())
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=10, after_scheduler=scheduler_cosine)

# Loss

In [14]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Source Loss', 'Train Acc.', 'Valid Acc.', 'Source ACC.', 'Train AUROC', 'Valid AUROC', 'Source AUROC'])

# Train

In [15]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    end = time.time()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            rand_index = torch.randperm(inputs.size()[0]).cuda()
            tt= targets[rand_index]
            boolean = targets==tt
            rand_index = rand_index[boolean]
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[boolean, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss =  loss_main
        loss_cls = 0
        loss_sp = 0

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
#         auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))
#         arc.update(auroc, inputs.size(0))
        cls_losses.update(loss_cls, inputs.size(0))
        sp_losses.update(loss_sp, inputs.size(0))
        main_losses.update(loss_main.tolist(), inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
#         if batch_idx % 100 == 0:
#             print('{batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
#                      batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    print('Train | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [16]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()


    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # compute output
            outputs = model(inputs)
            loss_main = criterion(outputs, targets)
            loss = loss_main
            loss_cls = 0
            loss_sp = 0

            # measure accuracy and record loss
            prec1 = accuracy(outputs.data, targets.data)
            auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
            losses.update(loss.data.tolist(), inputs.size(0))
            top1.update(prec1[0], inputs.size(0))
            arc.update(auroc, inputs.size(0))
            cls_losses.update(loss_cls, inputs.size(0))
            sp_losses.update(loss_sp, inputs.size(0))
            main_losses.update(loss_main.tolist(), inputs.size(0))


            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

    print('Test | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [17]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc, train_auroc = train(train_loader, student_model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc, test_auroc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
    source_loss, source_acc, source_auroc = test(val_source_loader, student_model, criterion, epoch, use_cuda)


    logger.append([state['lr'], train_loss, test_loss,  source_loss, train_acc, test_acc,source_acc, train_auroc, test_auroc, source_auroc])
    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : student_model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()


Epoch: [1 | 500] LR: 0.001000
Train | 20/20 | Loss:1.6064 | MainLoss:1.6064 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:53.7000 | AUROC:0.0000
Test | 153/20 | Loss:1.0019 | MainLoss:1.0019 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:55.5374 | AUROC:0.5799
Test | 161/20 | Loss:0.0038 | MainLoss:0.0038 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:99.9065 | AUROC:1.0000

Epoch: [2 | 500] LR: 0.001300
Train | 20/20 | Loss:0.8776 | MainLoss:0.8776 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:55.4250 | AUROC:0.0000
Test | 153/20 | Loss:0.7862 | MainLoss:0.7862 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:56.3434 | AUROC:0.5897
Test | 161/20 | Loss:0.0081 | MainLoss:0.0081 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:99.9346 | AUROC:1.0000

Epoch: [3 | 500] LR: 0.001600
Train | 20/20 | Loss:0.7556 | MainLoss:0.7556 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:55.8750 | AUROC:0.0000
Test | 153/20 | Loss:0.7179 | MainLoss:0.7179 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:56.8644 | AUROC:0.5982
Test | 161/20 | Loss:0.0151 | MainL

Train | 20/20 | Loss:0.5875 | MainLoss:0.5875 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:68.1750 | AUROC:0.0000
Test | 153/20 | Loss:0.5847 | MainLoss:0.5847 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:68.3912 | AUROC:0.7550
Test | 161/20 | Loss:0.0402 | MainLoss:0.0402 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:99.6044 | AUROC:1.0000

Epoch: [25 | 500] LR: 0.003993
Train | 20/20 | Loss:0.5758 | MainLoss:0.5758 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:69.7000 | AUROC:0.0000
Test | 153/20 | Loss:0.5792 | MainLoss:0.5792 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:68.9810 | AUROC:0.7612
Test | 161/20 | Loss:0.0404 | MainLoss:0.0404 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:99.5234 | AUROC:1.0000

Epoch: [26 | 500] LR: 0.003992
Train | 20/20 | Loss:0.5755 | MainLoss:0.5755 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:68.8250 | AUROC:0.0000
Test | 153/20 | Loss:0.5724 | MainLoss:0.5724 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:69.5085 | AUROC:0.7685
Test | 161/20 | Loss:0.0406 | MainLoss:0.0406 | SPLoss:0.0000 | 


Epoch: [47 | 500] LR: 0.003952
Train | 20/20 | Loss:0.4660 | MainLoss:0.4660 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:77.5750 | AUROC:0.0000
Test | 153/20 | Loss:0.4794 | MainLoss:0.4794 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:76.5433 | AUROC:0.8495
Test | 161/20 | Loss:0.0581 | MainLoss:0.0581 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:98.3084 | AUROC:0.9999

Epoch: [48 | 500] LR: 0.003949
Train | 20/20 | Loss:0.4530 | MainLoss:0.4530 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:78.3000 | AUROC:0.0000
Test | 153/20 | Loss:0.4763 | MainLoss:0.4763 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:76.8054 | AUROC:0.8515
Test | 161/20 | Loss:0.0582 | MainLoss:0.0582 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:98.2741 | AUROC:0.9999

Epoch: [49 | 500] LR: 0.003946
Train | 20/20 | Loss:0.4644 | MainLoss:0.4644 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:77.8250 | AUROC:0.0000
Test | 153/20 | Loss:0.4731 | MainLoss:0.4731 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:76.9332 | AUROC:0.8537
Test | 161/20 | Loss:0.0619 | Ma

Train | 20/20 | Loss:0.4402 | MainLoss:0.4402 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:78.7250 | AUROC:0.0000
Test | 153/20 | Loss:0.4636 | MainLoss:0.4636 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:77.7163 | AUROC:0.8613
Test | 161/20 | Loss:0.0654 | MainLoss:0.0654 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:97.8287 | AUROC:0.9999

Epoch: [71 | 500] LR: 0.000386
Train | 20/20 | Loss:0.4310 | MainLoss:0.4310 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:80.3250 | AUROC:0.0000
Test | 153/20 | Loss:0.4635 | MainLoss:0.4635 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:77.7621 | AUROC:0.8614
Test | 161/20 | Loss:0.0660 | MainLoss:0.0660 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:97.8069 | AUROC:0.9999

Epoch: [72 | 500] LR: 0.000386
Train | 20/20 | Loss:0.4364 | MainLoss:0.4364 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:78.7250 | AUROC:0.0000
Test | 153/20 | Loss:0.4633 | MainLoss:0.4633 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:77.7752 | AUROC:0.8616
Test | 161/20 | Loss:0.0660 | MainLoss:0.0660 | SPLoss:0.0000 | 

Train | 20/20 | Loss:0.4236 | MainLoss:0.4236 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:80.1000 | AUROC:0.0000
Test | 153/20 | Loss:0.4582 | MainLoss:0.4582 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:78.0210 | AUROC:0.8656
Test | 161/20 | Loss:0.0689 | MainLoss:0.0689 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:97.6168 | AUROC:0.9999

Epoch: [94 | 500] LR: 0.000374
Train | 20/20 | Loss:0.4342 | MainLoss:0.4342 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:80.1500 | AUROC:0.0000
Test | 153/20 | Loss:0.4579 | MainLoss:0.4579 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:78.0374 | AUROC:0.8655
Test | 161/20 | Loss:0.0689 | MainLoss:0.0689 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:97.6231 | AUROC:0.9999

Epoch: [95 | 500] LR: 0.000373
Train | 20/20 | Loss:0.4226 | MainLoss:0.4226 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:80.2750 | AUROC:0.0000
Test | 153/20 | Loss:0.4577 | MainLoss:0.4577 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:78.0603 | AUROC:0.8654
Test | 161/20 | Loss:0.0690 | MainLoss:0.0690 | SPLoss:0.0000 | 

Train | 20/20 | Loss:0.4206 | MainLoss:0.4206 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:80.2750 | AUROC:0.0000
Test | 153/20 | Loss:0.4532 | MainLoss:0.4532 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:78.3519 | AUROC:0.8690
Test | 161/20 | Loss:0.0717 | MainLoss:0.0717 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:97.4611 | AUROC:0.9999

Epoch: [117 | 500] LR: 0.000358
Train | 20/20 | Loss:0.4214 | MainLoss:0.4214 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:80.3500 | AUROC:0.0000
Test | 153/20 | Loss:0.4531 | MainLoss:0.4531 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:78.3814 | AUROC:0.8689
Test | 161/20 | Loss:0.0727 | MainLoss:0.0727 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:97.3988 | AUROC:0.9999

Epoch: [118 | 500] LR: 0.000357
Train | 20/20 | Loss:0.4096 | MainLoss:0.4096 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:81.4000 | AUROC:0.0000
Test | 153/20 | Loss:0.4530 | MainLoss:0.4530 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:78.3879 | AUROC:0.8691
Test | 161/20 | Loss:0.0721 | MainLoss:0.0721 | SPLoss:0.0000 

Train | 20/20 | Loss:0.4048 | MainLoss:0.4048 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:80.8750 | AUROC:0.0000
Test | 153/20 | Loss:0.4490 | MainLoss:0.4490 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:78.5976 | AUROC:0.8720
Test | 161/20 | Loss:0.0746 | MainLoss:0.0746 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:97.2897 | AUROC:0.9998

Epoch: [140 | 500] LR: 0.000339
Train | 20/20 | Loss:0.4037 | MainLoss:0.4037 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:81.4750 | AUROC:0.0000
Test | 153/20 | Loss:0.4490 | MainLoss:0.4490 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:78.6140 | AUROC:0.8725
Test | 161/20 | Loss:0.0759 | MainLoss:0.0759 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:97.2337 | AUROC:0.9999

Epoch: [141 | 500] LR: 0.000338
Train | 20/20 | Loss:0.4021 | MainLoss:0.4021 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:81.2750 | AUROC:0.0000
Test | 153/20 | Loss:0.4488 | MainLoss:0.4488 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:78.6468 | AUROC:0.8722
Test | 161/20 | Loss:0.0748 | MainLoss:0.0748 | SPLoss:0.0000 